In [136]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from tabulate import tabulate
import re
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import os
import sys

module_path = '/home/serinatan/project/gpgpu-sim_simulations/util/job_launching/results'
if module_path not in sys.path:
    sys.path.append(module_path)
    
import helper.help_iso as hi
import helper.format as fmt


bench_dict = {'cut_sgemm-0':0, 'cut_sgemm-1':0, 'cut_wmma-0': 0, 'cut_wmma-1': 0, 
         'parb_stencil-0': 1, 'parb_sgemm-0': 0,
         'parb_lbm-0': 1, 'parb_spmv-0': 1, 'parb_cutcp-0': 0}

mpl.style.use('seaborn-paper')





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read CSVs

In [146]:
df_seq = pd.read_csv('seq.csv')
df_intra = pd.read_csv('intra.csv')
df_inter = pd.read_csv('inter.csv')
print(df_intra.columns)

Index(['pair_str', 'config', 'gpusim_version', 'jobId', 'grid_x', 'grid_y',
       'grid_z', 'block_x', 'block_y', 'block_z', 'ctas/SM', 'runtime', 'ipc',
       'instructions', 'avg_mem_lat', 'avg_core_to_l2', 'avg_l2_to_core',
       'avg_mrq_latency', 'stall_core_ldst', 'stall_icnt_to_l2', 'l2_BW',
       'l2_rshr_entry_fail', 'l2_rshr_merge_fail', 'stall_l2_to_icnt',
       'l1D_miss_rate', 'l2_miss_rate', 'l2_total_accesses', 'packet_lat_out',
       'network_lat_out', 'inject_out', 'accepted_out', 'packet_lat_in',
       'network_lat_in', 'inject_in', 'accepted_in',
       'mem_subpartition_parallism', 'mem_subpartition_parallism_util',
       'L2_reservation_fail', 'empty_warp', 'stall_warp', 'idle_warp',
       'scoreboard_warp', 'regs', 'smem', 'dram_eff', 'dram_bw',
       'row_buffer_locality', 'mrqq', 'total_cmd', 'wasted_col', 'wasted_row',
       'mem_idle', 'CCDLc', 'WTRc', 'RTWc', 'RCDc', 'RCDWRc'],
      dtype='object')


In [153]:
# scale all IPC to baseline
baseline_ipc = pd.Series(df_seq.ipc.values,index=df_seq.pair_str).to_dict() 
baseline_runtime = pd.Series(df_seq.runtime.values,index=df_seq.pair_str).to_dict()

def norm_ipc_row(row, ipc='ipc', bench='pair_str'):
    return row[ipc] / baseline_ipc[row[bench]]

def norm_runtime_row(row, runtime='runtime', bench='pair_str'):
    return baseline_runtime[row[bench]] / row[runtime]

## Process columns

In [154]:
df_intra['norm_ipc'] = df_intra.apply(lambda row: norm_ipc_row(row), axis=1)
df_inter['norm_ipc'] = df_inter.apply(lambda row: norm_ipc_row(row), axis=1)

# decompose gpgpu-sim configs
hi.process_config_column('intra', 'l2', df=df_intra)
hi.process_config_column('inter', 'l2', df=df_inter)

df_intra['avg_dram_bw'] = df_intra['dram_bw'].transform(hi.avg_array)
df_inter['avg_dram_bw'] = df_inter['dram_bw'].transform(hi.avg_array)

df_intra['dram_busy'] = 1 - np.divide(df_intra['mem_idle'].transform(hi.avg_array),
                                      df_intra['total_cmd'].transform(hi.avg_array))
df_inter['dram_busy'] = 1 - np.divide(df_inter['mem_idle'].transform(hi.avg_array),
                                      df_inter['total_cmd'].transform(hi.avg_array))

idle_sum = df_intra[['empty_warp', 'idle_warp']].sum(axis=1)
df_intra['comp_busy'] = 1 - idle_sum / (idle_sum + df_intra[['stall_warp', 'scoreboard_warp']].sum(axis=1))

idle_sum = df_inter[['empty_warp', 'idle_warp']].sum(axis=1)
df_inter['comp_busy'] = 1 - idle_sum / (idle_sum + df_inter[['stall_warp', 'scoreboard_warp']].sum(axis=1))

max_cta_volta = 32
max_thread_volta = 2048
max_smem = 96*1024
max_register = 64*1024

df_intra['cta_ratio'] = df_intra['intra'] / max_cta_volta
threads = df_intra['intra'] * df_intra['block_x'] * df_intra['block_y'] * df_intra['block_z']
df_intra['thread_ratio'] = threads / max_thread_volta
df_intra['smem_ratio'] = df_intra['intra'] * df_intra['smem'] / max_smem
df_intra['reg_ratio'] = threads * df_intra['regs'] / max_register
# df_intra['dominant_resc'] = df_intra[['cta_ratio', 'thread_ratio', 'smem_ratio', 'reg_ratio']].max(axis=1)

def pow_2(*resc_list):
    done_first = False
    for r in resc_list:
        if done_first:
            usage = usage + df_intra[r] ** 2
        else:
            done_first = True
            usage = df_intra[r] ** 2
    return usage
            
df_intra['usage'] = pow_2('cta_ratio', 'thread_ratio', 'smem_ratio', 'reg_ratio', 'l2', 'dram_busy', 'comp_busy')

print(df_intra.columns)

Index(['pair_str', 'config', 'gpusim_version', 'jobId', 'grid_x', 'grid_y',
       'grid_z', 'block_x', 'block_y', 'block_z', 'ctas/SM', 'runtime', 'ipc',
       'instructions', 'avg_mem_lat', 'avg_core_to_l2', 'avg_l2_to_core',
       'avg_mrq_latency', 'stall_core_ldst', 'stall_icnt_to_l2', 'l2_BW',
       'l2_rshr_entry_fail', 'l2_rshr_merge_fail', 'stall_l2_to_icnt',
       'l1D_miss_rate', 'l2_miss_rate', 'l2_total_accesses', 'packet_lat_out',
       'network_lat_out', 'inject_out', 'accepted_out', 'packet_lat_in',
       'network_lat_in', 'inject_in', 'accepted_in',
       'mem_subpartition_parallism', 'mem_subpartition_parallism_util',
       'L2_reservation_fail', 'empty_warp', 'stall_warp', 'idle_warp',
       'scoreboard_warp', 'regs', 'smem', 'dram_eff', 'dram_bw',
       'row_buffer_locality', 'mrqq', 'total_cmd', 'wasted_col', 'wasted_row',
       'mem_idle', 'CCDLc', 'WTRc', 'RTWc', 'RCDc', 'RCDWRc', 'norm_ipc',
       'intra', 'l2', 'avg_dram_bw', 'dram_busy', 'comp_busy

In [129]:
def print_intra(df, benchmark):
    filename = '{0}-{1}.pdf'.format(benchmark, 'intra')
    filename = os.path.join('plots', filename)
    with PdfPages(filename) as pdf:
        hi.plot_page_intra(df, 'norm_ipc', benchmark, pdf)
        hi.plot_page_intra(df, 'avg_dram_bw', benchmark, pdf)
        hi.plot_page_intra(df, 'dram_busy', benchmark, pdf)
        hi.plot_page_intra(df, 'l2_miss_rate', benchmark, pdf)
        hi.plot_page_intra(df, 'l2_BW', benchmark, pdf)
        hi.plot_page_intra(df, 'l2_total_accesses', benchmark, pdf)
        hi.plot_page_intra(df, 'l1D_miss_rate', benchmark, pdf)
        hi.plot_page_intra(df, 'avg_mem_lat', benchmark, pdf)

def print_intra_inter(df_intra, df_inter, benchmark):
    filename = '{0}-{1}.pdf'.format(benchmark, 'both')
    filename = os.path.join('plots', filename)
    with PdfPages(filename) as pdf:
        hi.plot_page_intra_inter(df_intra, df_inter, 'norm_ipc', benchmark, pdf)
 

In [78]:
print_intra_inter(df_intra, df_inter, 'parb_stencil-0')

In [79]:
print_intra_inter(df_intra, df_inter, 'parb_cutcp-0')

## Intra SM Only

In [42]:
bench_list = list(df_intra['pair_str'].unique())
bench_list.sort()

In [35]:
for bench in bench_list:
    print(bench)
    print_intra(df_intra, bench)

cut_sgemm-1
cut_wmma-1
parb_stencil-0
parb_spmv-0
cut_sgemm-0
parb_cutcp-0
parb_lbm-0
cut_wmma-0


In [51]:
fig_tot, axs = plt.subplots(2, 4, figsize=(40, 30))   
axs = axs.flat

for ax, bench in zip(axs, bench_list):
    _df = df_intra[df_intra['pair_str'] == bench]
    
    hi.plot_heatmap(_df, x_key='intra', y_key='l2', z_key='norm_ipc', title=bench, axis=ax, scale=1.2)


fig_tot.suptitle('Intra, Normalized IPC', fontsize=18)
fig_tot.savefig('plots/total.pdf')
plt.close()




In [137]:
df_intra['perfdollar'] = df_intra['norm_ipc'] / df_intra['usage']
cols = ['pair_str', 'perfdollar', 'intra', 'norm_ipc', 'usage', 'l2', 'cta_ratio', 
        'thread_ratio', 'smem_ratio', 'reg_ratio', 'comp_busy', 'dram_busy']
sort = df_intra[cols].sort_values(['pair_str', 'perfdollar'], ascending=[True, True])
sort = sort[sort['norm_ipc'] > 0.8]

best_df = []
for bench in bench_list:
    idx = df_intra[(df_intra['norm_ipc'] > 0.8) & (df_intra['pair_str'] == bench)]['perfdollar'].idxmax()
    best_df.append(df_intra.iloc[idx])
    
best_df = pd.concat(best_df, axis=1).T[cols]

# formatting
pd.options.display.float_format = '{:,}'.format
    
best_df.style.set_table_styles(fmt.table_style).hide_index()\
            .format({'norm_ipc': "{:.4f}", 'dominant_resc': '{:.2f}', 
                     
                    })\

# print(best_df)

KeyError: 'norm_ipc'

## Shared

In [156]:
df_pair = pd.read_csv('pair.csv')
print(df_pair.columns)

# split pair into two benchmarks
pair = [re.split(r'-(?=\D)', p) for p in df_pair['pair_str']]
df_bench = pd.DataFrame(pair, columns=['bench_1', 'bench_2'])
df_pair = pd.concat([df_bench, df_pair], axis=1)

# extract resource allocation size
hi.process_config_column('intra', 'intra_2', 'l2', 'l2_2', df=df_pair)

# calculate slowdown
df_pair['slowdown_1'] = df_pair.apply(lambda row: norm_runtime_row(row, runtime='1_runtime', bench='bench_1'), 
                                      axis=1)
df_pair['slowdown_2'] = df_pair.apply(lambda row: norm_runtime_row(row, runtime='2_runtime', bench='bench_2'), 
                                      axis=1)

# display in a table
display_cols = ['bench_1', 'bench_2', 'intra', 'intra_2', 'l2', 'l2_2', 'slowdown_1', 'slowdown_2']
df_pair[display_cols].style.set_table_styles(fmt.table_style).hide_index()

Index(['pair_str', 'config', 'gpusim_version', 'jobId', '1_grid_x', '1_grid_y',
       '1_grid_z', '1_block_x', '1_block_y', '1_block_z', '1_ctas/SM',
       '1_runtime', '1_instructions', '1_avg_mem_lat', '1_avg_core_to_l2',
       '1_avg_l2_to_core', '1_avg_mrq_latency', '2_grid_x', '2_grid_y',
       '2_grid_z', '2_block_x', '2_block_y', '2_block_z', '2_ctas/SM',
       '2_runtime', '2_instructions', '2_avg_mem_lat', '2_avg_core_to_l2',
       '2_avg_l2_to_core', '2_avg_mrq_latency', 'stall_icnt_to_l2',
       'stall_l2_to_icnt', 'stall_core_ldst', 'l1D_miss_rate', 'l2_miss_rate',
       'runtime', 'l2_rshr_entry_fail', 'l2_rshr_merge_fail', 'dram_bw'],
      dtype='object')


bench_1,bench_2,intra,intra_2,l2,l2_2,slowdown_1,slowdown_2
cut_wmma-0,parb_lbm-0,2,3,0.5,0.5,0.24173,0.849939
cut_wmma-0,parb_stencil-0,2,3,0.5,0.5,0.798147,0.440257
cut_sgemm-1,parb_spmv-0,1,6,0.5,0.5,0.962535,0.864206
cut_sgemm-1,parb_stencil-0,1,3,0.5,0.5,0.730872,0.761517
cut_wmma-0,parb_spmv-0,2,6,0.5,0.5,0.671119,0.508979
